# Load Data: Handling badly CSV file

In this notebook, I show how to load a badly formatted CSV file. <br>
You can find the dataset either in the data folder or [here](https://www.kaggle.com/datasets/sahirmaharajj/restaurant-inspection-results).

## Table-of-Content

* [Try Loading Data](#try-load)
* [Load Good Cases](#load-goods)
* [Handle Bad Cases](#handle-bad)
  * [Load bad cases](#load-bads)
  * [Strategy to handle bad cases](#strategy-handle)
* [Final Notes](#final-notes)

In [27]:
import csv
import warnings
import pandas as pd

warnings.filterwarnings("ignore")

## Try Loading CSV file <a class="anchor" id="try-load"></a>

If you try reading the file normally as shown below, you get an error message.
```python
filename = "data/DOHMH_New_York_City_Restaurant_Inspection_Results_20240429.csv"
data = pd.read_csv(filename)
```
> **ParserError: Error tokenizing data. C error: Expected 27 fields in line 1155, saw 869**

This is just one of the possible errors, if you skip bad lines as shown in the example below, you are still get another error.

```python
data = pd.read_csv(filename, on_bad_lines='skip')
```
> **ParserError: Error tokenizing data. C error: EOF inside string starting at row 4587**

After checking the file using Excel (or LibreOffice Calc), you might also see that the file has some inconsistencies.<br> 
One of them is related to **double quotes**. They were not placed correctly. Then, you try again, but setting the parameter **quoting**.
```python
import csv
data = pd.read_csv(filename, on_bad_lines='skip', quoting=csv.QUOTE_NONE)
```

**It finally loads the data**, but if you pay attention to the number of rows, <br>
it read much less compared to the total amount of rows (i.e. use Excel or use dummy separator to get everything into one column)

```python
# Load data into one Columns
one_column_data = pd.read_csv(filename, sep="delimiter", names=['col1'])
```

Therefore, we need to find another way to handle the badly formatted cases. Otherwise, we lose important information.

## Load Good Cases <a class="anchor" id="load-goods"></a>

In [28]:
filename = "data/DOHMH_New_York_City_Restaurant_Inspection_Results_20240429.csv"

# Non-problematic cases
good_cases = pd.read_csv(filename, on_bad_lines='skip', 
                         quoting=csv.QUOTE_NONE, encoding='utf-8')

display( good_cases.head() )

,CAMIS,DBA,BORO,BUILDING,STREET,ZIPCODE,PHONE,CUISINE DESCRIPTION,INSPECTION DATE,ACTION,...,INSPECTION TYPE,Latitude,Longitude,Community Board,Council District,Census Tract,BIN,BBL,NTA,Location Point1;;;
0,50105376,H H HEAT WAVE,Queens,8812,ROCKAWAY BEACH BLVD,11693.0,3475007435,NaN,01/01/1900,NaN,...,NaN,40.587276,-73.813243,414.0,32.0,94202.0,4448991.0,4.161230e+09,QN12,;;;
1,50148665,Lucille's,Manhattan,1415,2 AVENUE,10021.0,9179233223,NaN,01/01/1900,NaN,...,NaN,40.769924,-73.957554,108.0,4.0,12600.0,1043999.0,1.014280e+09,MN31,;;;
2,50149698,NaN,Brooklyn,7101,FORT HAMILTON PARKWAY,11228.0,3477754169,NaN,01/01/1900,NaN,...,NaN,40.627645,-74.013672,310.0,43.0,20600.0,3147032.0,3.059020e+09,BK30,;;;
3,50150122,NaN,Queens,11011,101ST AVE,11419.0,8608537003,NaN,01/01/1900,NaN,...,NaN,40.687852,-73.833405,409.0,28.0,11800.0,4197955.0,4.094120e+09,QN54,;;;
4,50134862,THE CHOCOLATE FACTORY,Brooklyn,70,SCOTT AVENUE,11237.0,6462440734,NaN,01/01/1900,NaN,...,NaN,40.709853,-73.922706,301.0,34.0,44900.0,3070597.0,3.029900e+09,BK90,;;;


## Handle Bad Cases <a class="anchor" id="handle-bad"></a>

After loading the good cases, it is important to note that the **total number of features is 27**. Therefore, we should have the same number of features in the bad cases.<br>
Also, the there are commas inside double quotes. These are correctly placed, but when we use comma as a separator. We end up with more features than we need. <br>
Therefore, we should consider avoid splitting these strings.

The initial steps are:
- Load the data into one column.
- Select problematic rows/lines* 
  - The problematic rows are the ones with **double quotes**.

### Load bad cases <a class="anchor" id="load-bads"></a>

In [31]:
# Load data into one column
ns_data = pd.read_csv(filename, sep="delimiter", names=['col1'])

#. Select problematic rows/lines (double quotes)
bad_cases = ns_data[ns_data['col1'].str.contains('\"')]

display( bad_cases.head() )

,col1
75,"""40958170,SOHO HOUSE,Manhattan,2935,9 AVENUE,,..."
118,"""41395395,""""CITI FIELD BUD ISLAND, STAND 140""""..."
174,"""40400739,STARBUCKS,Manhattan,1325,ASTOR PLACE..."
176,"""50090491,""""FORT WASHINGTON PUBLIC HOUSE, BROA..."
329,"""41395407,""""CITI FIELD BASH BURGER, STAND 135""..."


### Strategy to handle bad cases <a class="anchor" id="strategy-handle"></a>

After checking the bad cases (i.e. see cases above), one way to handle that is described as follows:

1. Replace **double-double quotes** to simple ones.
2. Find the **first** and **last** double quotes.
3. Remove wrongly placed double quotes: **skip first double quote (by sliding) and replace the last one with None.**
4. Split string by taking quotechar into consideration (i.e. there are some commas in these double quotes).
5. Convert results into a DataFrame.
6. Drop uncessary features (i.e. three features).
7. Merge datasets (good cases and converted ones).
8. Set feature names based on the good dataset.

In [32]:
def split_bad_cases(row):
    """
    This function handles badly formatted data and 
    splits into correct features.
    """
    
    #. Get String
    row = row.iloc[0]
    
    #. Replace double-double quotes to only one double quote.
    row = row.replace('""', '"')
    
    #. Find wrong placed double-quotes
    first_dq = row.find('"')
    last_dq = row.rfind('"')
    
    #. Remove wrong double qotes by skipping-sliding.
    row = row[first_dq+1:last_dq] + row[last_dq:].replace('"', "")
    
    #. Split string but considering commas inside double quotes
    splitted_data = [ '{}'.format(x) for x in list(
        csv.reader([row], delimiter=',', quotechar='"'))[0] ]

    return splitted_data

In [34]:
#. Handle bad cases: splitting each row
converted_rows = bad_cases.apply(split_bad_cases, axis=1)

#. Convert results into a DataFrame
structured_data = pd.DataFrame(converted_rows.to_list())

#. Drop uncessary features (the last three features)
converted_df = structured_data.iloc[:,:-3]

#. Set feature names
converted_df.columns = good_cases.columns

#. Merge datasets
dataset = pd.concat([good_cases, converted_df])

display( dataset )

,CAMIS,DBA,BORO,BUILDING,STREET,ZIPCODE,PHONE,CUISINE DESCRIPTION,INSPECTION DATE,ACTION,...,INSPECTION TYPE,Latitude,Longitude,Community Board,Council District,Census Tract,BIN,BBL,NTA,Location Point1;;;
0,50105376,H H HEAT WAVE,Queens,8812,ROCKAWAY BEACH BLVD,11693.0,3475007435,NaN,01/01/1900,NaN,...,NaN,40.587276,-73.813243,414.0,32.0,94202.0,4448991.0,4161230094.0,QN12,;;;
1,50148665,Lucille's,Manhattan,1415,2 AVENUE,10021.0,9179233223,NaN,01/01/1900,NaN,...,NaN,40.769924,-73.957554,108.0,4.0,12600.0,1043999.0,1014280027.0,MN31,;;;
2,50149698,NaN,Brooklyn,7101,FORT HAMILTON PARKWAY,11228.0,3477754169,NaN,01/01/1900,NaN,...,NaN,40.627645,-74.013672,310.0,43.0,20600.0,3147032.0,3059020010.0,BK30,;;;
3,50150122,NaN,Queens,11011,101ST AVE,11419.0,8608537003,NaN,01/01/1900,NaN,...,NaN,40.687852,-73.833405,409.0,28.0,11800.0,4197955.0,4094120047.0,QN54,;;;
4,50134862,THE CHOCOLATE FACTORY,Brooklyn,70,SCOTT AVENUE,11237.0,6462440734,NaN,01/01/1900,NaN,...,NaN,40.709853,-73.922706,301.0,34.0,44900.0,3070597.0,3029900001.0,BK90,;;;
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5263,50106034,TACO BELL,Staten Island,1281,FOREST AVENUE,10302,9089072910,Tex-Mex,10/01/2021,Violations were cited in the following area(s).,...,Pre-permit (Operational) / Initial Inspection,40.626363812547,-74.132455470484,501,49,020100,5024490,5010420013,SI07,;;;
5264,50102906,KRISPY KREME,Bronx,371,EAST FORDHAM ROAD,10458,9292413068,Coffee/Tea,09/27/2023,Violations were cited in the following area(s).,...,Cycle Inspection / Initial Inspection,40.861857338919,-73.892185387035,207,15,039901,2016303,2032750001,BX05,;;;
5265,50056212,VIA ROMA PIZZA BAR,Brooklyn,445,COURT STREET,11231,9293379714,Pizza,03/01/2024,Violations were cited in the following area(s).,...,Cycle Inspection / Re-inspection,40.678034015081,-73.997970234948,306,39,007700,3008013,3004670002,BK33,;;;
5266,40371727,LA PALINA RESTAURANT,Brooklyn,159,AVENUE O,11204,7182369764,Italian,12/22/2021,Violations were cited in the following area(s).,...,Cycle Inspection / Re-inspection,40.610776716763,-73.978869293745,311,44,043400,3173432,3065800034,BK29,;;;


## Final Notes <a class="anchor" id="final-notes"></a>

During any Data Science project, we must remove any problem with the data such as inconsistency before starting with the analysis. <br>
Otherwise, this affects negatively the analytical process as well as it leads to wrong results.

After handling the bad cases, we are ready to analyze the data. If we decided to go only with the good cases, <br>
we would have lose a lot of information Then, it was important to handle these bad cases.



___